File used for testing other components main files begin with the numbers 1-5 respectively

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key="", model="embed-english-v3.0", user_agent='langchain')

with open(f'er_text_documents/dialogue.txt', errors='ignore') as f:
    public_world_info = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 500,
    length_function = len,
    separators = ['\n\n\n', '\n\n', '\n', '.', ',', ' '],
    is_separator_regex=False,
)
documents = text_splitter.create_documents([public_world_info])
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="vectordbs\public_world_info",
)
vector_store.add_documents(documents=documents, ids=uuids)

In [ ]:
from tools.audio_recorder import AudioRecorder
from tools.audio_transcriber import AudioTranscriber
from tools.ai_chatbot_base import AIChatbotBase
from tools.open_voice_tts import OpenVoiceTTS
from playsound import playsound
import keyboard

def listen_for_key():
    recorder = AudioRecorder()
    transcriber = AudioTranscriber()
    chatbot = AIChatbot(api_key_file='api.txt')  # Instantiate AIChat
    open_voice_tts = OpenVoiceTTS()

    try:
        print("Press and hold the spacebar to record.")
        print("Release the spacebar to stop recording.")
        print("Press Esc to terminate the program.")

        while True:
            if keyboard.is_pressed('space'):
                if not recorder.recording:
                    recorder.start_recording()

            elif recorder.recording:
                #record audio
                recorder.stop_recording()
                #transcribe audio
                transcribed_text = transcriber.transcribe_audio(recorder.OUTPUT_FILENAME)

                #pass message to ai
                ai_response = chatbot.ask(transcribed_text, "Annie")
                print("AI Response:", ai_response)
                
                #convert response to voice
                reference_speaker_file = 'OpenVoice/resources/demo_speaker1.mp3'
                target_se, audio_name = open_voice_tts.extract_target_speaker(reference_speaker_file)
                open_voice_tts.generate_audio(text=ai_response, target_se=target_se)
                
                #play sound
                # audio_file_path = 'OpenVoice/outputs/output_en_default.wav'
                # playsound(audio_file_path)
                
            if keyboard.is_pressed('esc'):
                if recorder.recording:
                    recorder.stop_recording()
                print("Terminating program...")
                break
    finally:
        recorder.cleanup()

In [ ]:
if __name__ == "__main__":
    listen_for_key()

In [ ]:
from tools.ai_chatbot import AIChatbot

chatbot = AIChatbot(api_key_file='api.txt',game='elden_ring',character='white_mask_varre')

In [ ]:
from playsound import playsound
import os
           
audio_file_path = 'OpenVoice/outputs/output_en_default.wav'

if os.path.exists(audio_file_path):
    playsound(audio_file_path)
else:
    print("Audio file does not exist.")